In [ ]:

# Install required libraries
!pip install numpy pandas scikit-learn tensorflow keras yfinance matplotlib seaborn


The main

In [ ]:
import os
import warnings
import yfinance as yf
import pandas as pd
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Suppress warnings to clean up output
warnings.filterwarnings("ignore")

# Stock Ticker Lists
NIFTY50_TICKERS = [
    'ADANIENT.ns', 'ADANIPORTS.ns', 'APOLLOHOSP.ns', 'ASIANPAINT.ns', 
    'AXISBANK.ns', 'BAJAJ-AUTO.ns', 'BAJFINANCE.ns', 'BAJAJFINSV.ns', 
    # ... (rest of Nifty 50 tickers)
]

SP500_TICKERS = [
    "AAPL", "MSFT", "AMZN", "GOOGL", "GOOG", "META", 
    # ... (rest of S&P 500 tickers)
]

# Configuration Parameters
START_DATE = '2022-07-26'
END_DATE = '2024-07-28'
PREDICTIONS_FOLDER = 'predictions'

def create_predictions_directory():
    """Create predictions directory if it doesn't exist."""
    if not os.path.exists(PREDICTIONS_FOLDER):
        os.makedirs(PREDICTIONS_FOLDER)

def initialize_csv_file(file_name):
    """Initialize a CSV file with standard prediction columns."""
    columns = ['Date', 'Name', 'Act', 'Pred_LR', 'Pred_RF', 'Pred_SVR', 'Pred_LSTM']
    df = pd.DataFrame(columns=columns)
    df.to_csv(file_name, index=False)

def append_prediction_to_csv(file_name, row_data):
    """Append a new prediction row to the CSV file."""
    if not os.path.exists(file_name):
        initialize_csv_file(file_name)
    
    df = pd.read_csv(file_name)
    df.loc[len(df)] = row_data
    df.to_csv(file_name, index=False)

def download_stock_data(tickers, start_date, end_date):
    """Download historical stock data for given tickers."""
    for ticker in tickers:
        file_name = f"data/{ticker}_historical_data.csv"
        print(f"Downloading data for {ticker}", end='\r')
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        stock_data.to_csv(file_name)
        print(f"Data for {ticker} saved to {file_name}")

def train_lstm_model(X, y):
    """Train an LSTM model for stock price prediction."""
    model_lstm = Sequential([
        LSTM(units=64, input_shape=(X.shape[1], 1)),
        Dense(1)
    ])
    model_lstm.compile(optimizer='adam', loss='mean_squared_error')
    model_lstm.fit(X.reshape(len(X), X.shape[1], 1), y, epochs=50, batch_size=32, verbose=0)
    return model_lstm

def predict_with_lstm(model, X_np):
    """Make a prediction using the trained LSTM model."""
    return model.predict(X_np.reshape(1, X_np.shape[0], 1), verbose=0)[0][0]

def prepare_stock_predictions():
    """Main function to generate stock predictions using multiple models."""
    create_predictions_directory()
    
    #unique symbols
    symbols = list(set(NIFTY50_TICKERS))
    
    for ticker in symbols:
        print(ticker, end='\r')
        file_name = f"data/{ticker}_historical_data.csv"
        df = pd.read_csv(file_name)
        
        if '2023-07-25' in list(df['Date']):
            idx_start = df.index[df['Date'] == '2023-07-25'].tolist()[0]
            
            # Compute initial LSTM model
            initial_df = df.iloc[:idx_start]
            initial_X = initial_df[['Open', 'High', 'Low', 'Close', 'Volume']]
            initial_y = (((initial_df['Low'] - initial_df['Open']) / initial_df['Open']) * 100).shift(-1)
            
            model_lstm = train_lstm_model(initial_X.values, initial_y[:idx_start].values)
            
            # Predict for each subsequent date
            for j in range(idx_start, len(df)):
                current_df = pd.read_csv(file_name)
                date = current_df['Date'].iloc[j]
                current_df['y'] = (((current_df['Low'] - current_df['Open']) / current_df['Open']) * 100).shift(-1)

                y = current_df['y']
                X_data = current_df[['Open', 'High', 'Low', 'Close', 'Volume']]
                
                X_train = X_data.iloc[:j]
                x_current = X_data.iloc[j]
                actual_value = y.iloc[j]
                
                # Train and predict with multiple models
                model_lr = LinearRegression().fit(X_train, y[:j])
                pred_lr = model_lr.predict([x_current])[0]
                
                model_rf = RandomForestRegressor(n_estimators=100).fit(X_train, y[:j])
                pred_rf = model_rf.predict([x_current])[0]
                
                model_svr = SVR(kernel='rbf').fit(X_train, y[:j])
                pred_svr = model_svr.predict([x_current])[0]
                
                pred_lstm = predict_with_lstm(model_lstm, x_current.values)
                
                # Prepare and save results
                results = [date, ticker, actual_value, pred_lr, pred_rf, pred_svr, pred_lstm]
                date_file_name = os.path.join(PREDICTIONS_FOLDER, f"{date}_predictions_nifty50.csv")
                append_prediction_to_csv(date_file_name, results)

if __name__ == "__main__":
    prepare_stock_predictions()

Backtester

In [ ]:
import os
import pandas as pd
from statistics import mean

def analyze_model_performance(predictions_folder="predictions"):
    """
    Analyze performance of different stock prediction models
    
    Args:
        predictions_folder (str): Path to folder containing prediction files
    """
    # Find all prediction files
    prediction_files = [
        f for f in os.listdir(predictions_folder) 
        if f.endswith('_predictions_nifty50.csv')
    ]
    
    # Models to evaluate
    model_columns = ['Pred_LR', 'Pred_RF', 'Pred_SVR', 'Pred_LSTM']
    
    # Performance analysis for each model
    for model in model_columns:
        acts = []
        account_balance = 100
        
        for file in prediction_files:
            file_path = os.path.join(predictions_folder, file)
            df = pd.read_csv(file_path)
            
            # Sort by model's prediction column
            df = df.sort_values(model, ascending=True).reset_index(drop=True)
            
            # Evaluate trading strategy
            if float(df['Act'][0]) <= -1:
                acts.append(1)  # Win
                account_balance *= 1.01  # Add 1%
            else:
                acts.append(0)  # Loss
                account_balance *= 0.99  # Subtract 1%
        
        # Calculate and print performance metrics
        win_rate = mean(acts)
        change = ((account_balance - 100) / 100) * 100
        
        print(f'Model: {model}')
        print(f'Win Rate: {win_rate:.2f}')
        print(f'Account Balance Change annually: {change:.2f}%\n')

if __name__ == "__main__":
    analyze_model_performance()